# Create Summary IMEI with Python

Make sure the column names as follows:
1. date_time → Date with format (yyyymmdd), ex: 19450817. Or with format (yyyymmddhh), ex: 1945081708
2. imei → Number of International Mobile Equipment Identity (IMEI)
3. fiveg → Description of IMEI that support 5G
- yes = IMEI support 5G
- no = IMEI not support 5G
4. kecamatan → Name of kecamatan

In [1]:
# Import required libraries
import glob
import pandas as pd

In [2]:
# Read raw data of IMEI and do data cleaning
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Raw data of IMEI is a ZIP extension, where each zip has a CSV file inside
file = glob.glob("*.zip")
df = [pd.read_csv(f, sep=',', skiprows=[0]) for f in file]
df = pd.concat(df, ignore_index=True)
df.head(5)

,date_time,msisdn,imei,fiveg,lac,ci,kabupaten,kecamatan
0,2022020721,62813811*****,359120542*****,no,144648,24,KOTA DEPOK,SAWANGAN
1,2022020721,62813860*****,865904040*****,no,144463,31,KOTA DEPOK,SUKMA JAYA
2,2022020721,62851551*****,865073054*****,no,144364,31,KOTA DEPOK,SUKMA JAYA
3,2022020721,62812911*****,352160551*****,no,144228,14,KOTA DEPOK,TAPOS
4,2022020721,62822136*****,868880042*****,no,144399,21,KOTA DEPOK,CIMANGGIS


In [3]:
# Convert a string to a title case in the 'kecamatan' column
df['kecamatan'] = df['kecamatan'].str.title()
df.head(5)

,date_time,msisdn,imei,fiveg,lac,ci,kabupaten,kecamatan
0,2022020721,62813811*****,359120542*****,no,144648,24,KOTA DEPOK,Sawangan
1,2022020721,62813860*****,865904040*****,no,144463,31,KOTA DEPOK,Sukma Jaya
2,2022020721,62851551*****,865073054*****,no,144364,31,KOTA DEPOK,Sukma Jaya
3,2022020721,62812911*****,352160551*****,no,144228,14,KOTA DEPOK,Tapos
4,2022020721,62822136*****,868880042*****,no,144399,21,KOTA DEPOK,Cimanggis


In [4]:
# Number of 4G users in Depok
total = df['fiveg'].count()
total

203409023

In [5]:
# Number of users who support 5G in Depok
sup5g = df.loc[df['fiveg'] == 'yes', ['fiveg']].shape[0]
sup5g

159468

In [6]:
# Percentage of users who support 5G in Depok (%)
sup5g_depok = (sup5g/total)*100
sup5g_depok

0.07839770215109877

In [7]:
# Name of kecamatan in Depok
kec = df['kecamatan'].unique()
kec

array(['Sawangan', 'Sukma Jaya', 'Tapos', 'Cimanggis', 'Cipayung',
       'Bojongsari', 'Limo', 'Pancoran Mas', 'Cinere', 'Cilodong', 'Beji'],
      dtype=object)

In [8]:
# Number of 4G users in each kecamatan in Depok
total_kec = []
for x in kec:
    a = df.loc[df['kecamatan'] == x].shape[0]
    total_kec.append(a)
total_kec

[18114809,
 22851049,
 24957017,
 28242279,
 12064767,
 9741476,
 10765855,
 23149146,
 13723800,
 18096969,
 21701856]

In [9]:
# Number of users who support 5G in each kecamatan in Depok
kec_sup5g = []
for x in kec:
    b = df.loc[df['fiveg'] == 'yes'].loc[df['kecamatan'] == x].shape[0]
    kec_sup5g.append(b)
kec_sup5g

[11159, 17324, 16858, 27162, 6586, 6227, 9968, 17962, 12407, 12391, 21424]

In [10]:
# Percentage of users who support 5G in each kecamatan in Depok (%)
per_kec_sup5g = [(x / y)*100 for x, y in zip(kec_sup5g, total_kec)]
per_kec_sup5g

[0.061601532756983525,
 0.07581271214288675,
 0.06754813686267074,
 0.0961749581186419,
 0.054588704448250015,
 0.06392255136695918,
 0.09258902335207003,
 0.07759249520479071,
 0.0904049898716099,
 0.06847002942868499,
 0.09871966711049968]

In [11]:
# Create DataFrame from dict of list
df2 = pd.DataFrame(list(zip(kec, kec_sup5g, per_kec_sup5g)), 
                   columns=['kecamatan_imei', 'user_support_5G', 'percentage_percent'])
df2

,kecamatan imei,user support 5G,percentage_percent
0,Sawangan,11159,0.061602
1,Sukma Jaya,17324,0.075813
2,Tapos,16858,0.067548
3,Cimanggis,27162,0.096175
4,Cipayung,6586,0.054589
5,Bojongsari,6227,0.063923
6,Limo,9968,0.092589
7,Pancoran Mas,17962,0.077592
8,Cinere,12407,0.090405
9,Cilodong,12391,0.068470


In [12]:
# Determine the recommended kecamatan for the development of 5G coverage in Depok
def recommendation(row):
    if row['percentage_percent'] >= sup5g_depok:
        val = 'Yes'
    else:
        val = 'No'
    return val

df2['recommended_imei'] = df2.apply(recommendation, axis=1)
df2

,kecamatan imei,user support 5G,percentage_percent,recommended imei
0,Sawangan,11159,0.061602,No
1,Sukma Jaya,17324,0.075813,No
2,Tapos,16858,0.067548,No
3,Cimanggis,27162,0.096175,Yes
4,Cipayung,6586,0.054589,No
5,Bojongsari,6227,0.063923,No
6,Limo,9968,0.092589,Yes
7,Pancoran Mas,17962,0.077592,No
8,Cinere,12407,0.090405,Yes
9,Cilodong,12391,0.068470,No


In [13]:
# Sort recommendation results from largest to smallest value
df2.sort_values(by=['percentage_percent'], inplace=True, ascending=False)
df2.insert(loc=0,column='no_imei', value = range(1, 1 + len(df2)))
df2

,no imei,kecamatan imei,user support 5G,percentage_percent,recommended imei
10,1,Beji,21424,0.098720,Yes
3,2,Cimanggis,27162,0.096175,Yes
6,3,Limo,9968,0.092589,Yes
8,4,Cinere,12407,0.090405,Yes
7,5,Pancoran Mas,17962,0.077592,No
1,6,Sukma Jaya,17324,0.075813,No
9,7,Cilodong,12391,0.068470,No
2,8,Tapos,16858,0.067548,No
5,9,Bojongsari,6227,0.063923,No
0,10,Sawangan,11159,0.061602,No


In [14]:
# Added 'time'column
time = str(df.iloc[0]['date_time'])
df2.insert(loc=1,column='time', value = time[0:6])
df2

,no imei,time,kecamatan imei,user support 5G,percentage_percent,recommended imei
10,1,202202,Beji,21424,0.098720,Yes
3,2,202202,Cimanggis,27162,0.096175,Yes
6,3,202202,Limo,9968,0.092589,Yes
8,4,202202,Cinere,12407,0.090405,Yes
7,5,202202,Pancoran Mas,17962,0.077592,No
1,6,202202,Sukma Jaya,17324,0.075813,No
9,7,202202,Cilodong,12391,0.068470,No
2,8,202202,Tapos,16858,0.067548,No
5,9,202202,Bojongsari,6227,0.063923,No
0,10,202202,Sawangan,11159,0.061602,No


In [15]:
# Saving recommendation results to CSV file
df2.to_csv('summary_imei.csv', index=False)

In [16]:
# Threshold IMEI, save this value to the database
sup5g_depok

0.07839770215109877